In [ ]:
from zipfile import ZipFile
import os
from tqdm import tqdm
from google.colab import drive
from google.colab.patches import cv2_imshow
import cv2
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
!pip install tflearn
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import matplotlib as plt
import numpy as np
from keras.models import Sequential
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D
from random import shuffle
from sklearn.model_selection import train_test_split
import os,cv2
from IPython.display import Image
from keras.preprocessing import image
from keras import optimizers
from keras import layers,models
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
import seaborn as sns
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from tensorflow.keras import optimizers
from keras.optimizers import rmsprop_v2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 28.1 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=f33d3c0b3403fbf6478a7e6960b5328ea5651ecb1481bf96ddd89d0ae0ea3144
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
drive.mount('/content/MyDrive/',force_remount = True)

Mounted at /content/MyDrive/


In [ ]:
!unzip '/content/MyDrive/MyDrive/NNDataset.zip'

Archive:  /content/MyDrive/MyDrive/NNDataset.zip
  inflating: Test/0.jpg              
  inflating: Test/1.jpg              
  inflating: Test/10.jpg             
  inflating: Test/100.jpg            
  inflating: Test/101.jpg            
  inflating: Test/102.jpg            
  inflating: Test/103.jpg            
  inflating: Test/104.jpg            
  inflating: Test/105.jpg            
  inflating: Test/106.jpg            
  inflating: Test/107.jpg            
  inflating: Test/108.jpg            
  inflating: Test/109.jpg            
  inflating: Test/11.jpg             
  inflating: Test/110.jpg            
  inflating: Test/111.jpg            
  inflating: Test/112.jpg            
  inflating: Test/113.jpg            
  inflating: Test/114.jpg            
  inflating: Test/115.jpg            
  inflating: Test/116.jpg            
  inflating: Test/117.jpg            
  inflating: Test/118.jpg            
  inflating: Test/119.jpg            
  inflating: Test/12.jpg             
 

In [ ]:
TRAIN_DIR = "/content/Train"
TEST_DIR = '/content/Test'
IMG_SIZE = 64
LR = 0.001
MODEL_NAME = 'AlexNet'

Basketball = []
Football = [] 
Rowing = []
Swimming = []
Tennis = []
Yoga = []

for img in tqdm(os.listdir(TRAIN_DIR)):
  path = os.path.join(TRAIN_DIR,img)
  image = cv2.imread(path,1)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
  if img.__contains__("Basketball"):
    Basketball.append([np.array(image), np.array([1,0,0,0,0,0])])
  elif img.__contains__("Football"):
    Football.append([np.array(image), np.array([0,1,0,0,0,0])])
  elif img.__contains__("Rowing"):
    Rowing.append([np.array(image), np.array([0,0,1,0,0,0])])
  elif img.__contains__("Swimming"):
    Swimming.append([np.array(image), np.array([0,0,1,0,0,0])])
  elif img.__contains__("Tennis"):
    Tennis.append([np.array(image), np.array([0,0,0,0,1,0])])
  elif img.__contains__("Yoga"):
    Yoga.append([np.array(image), np.array([0,0,0,0,0,1])])
print(" ")
print("number of each class")
print(len(Basketball))
print(len(Football))
print(len(Rowing))
print(len(Swimming))
print(len(Tennis))
print(len(Yoga))

####balance dataset by flip horizantal######
def augmentationFlipHorizontal(data,name):
  for img in data:
    if len(data) != 458:
      flappedImage = cv2.flip(image,1)
      if name == "Basketball":
        Basketball.append([np.array(flappedImage), np.array([1,0,0,0,0,0])])
      elif name == "Football":
        Football.append([np.array(flappedImage), np.array([0,1,0,0,0,0])])
      elif name == "Rowing":
        Rowing.append([np.array(flappedImage), np.array([0,0,1,0,0,0])])
      elif name == "Swimming":
        Swimming.append([np.array(flappedImage), np.array([0,0,0,1,0,0])])
      elif name == "Tennis":
        Tennis.append([np.array(flappedImage), np.array([0,0,0,0,1,0])])
      elif name == "Yoga":
        Yoga.append([np.array(flappedImage), np.array([0,0,0,0,0,1])])

augmentationFlipHorizontal(Basketball,"Basketball")
augmentationFlipHorizontal(Football,"Football")
augmentationFlipHorizontal(Rowing,"Rowing")
augmentationFlipHorizontal(Swimming,"Swimming")
augmentationFlipHorizontal(Tennis,"Tennis")
augmentationFlipHorizontal(Yoga,"Yoga")
print("number of each class after augmentation")
print(len(Basketball))
print(len(Football))
print(len(Rowing))
print(len(Swimming))
print(len(Tennis))
print(len(Yoga))
def create_label(image_name):
    """ Create an one-hot encoded vector from image name """
    word_label = image_name.split('_')[0]
    if word_label == 'Basketball':
        return np.array([1,0,0,0,0,0])
    elif word_label == 'Football':
        return np.array([0,1,0,0,0,0])
    elif word_label == 'Rowing':
        return np.array([0,0,1,0,0,0])
    elif word_label == 'Swimming':
        return np.array([0,0,0,1,0,0])
    elif word_label == 'Tennis':
        return np.array([0,0,0,0,1,0])
    elif word_label == 'Yoga':
        return np.array([0,0,0,0,0,1])


train = []
def createTrainData(data):
  for i in data:
    train.append(i)


createTrainData(Basketball)
createTrainData(Football)
createTrainData(Rowing)
createTrainData(Swimming)
createTrainData(Tennis)
createTrainData(Yoga)

shuffle(train)
print("shape of train data")
X_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y_train = np.array([i[1] for i in train])
print(X_train.shape)
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train ,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)

100%|██████████| 1681/1681 [00:08<00:00, 205.30it/s]

 
number of each class
196
400
202
240
185
458
number of each class after augmentation
458
458
458
458
458
458
shape of train data
(2748, 64, 64, 3)


In [ ]:
ops.reset_default_graph()
conv_input = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')
conv1 = conv_2d(conv_input, 32, 5, activation='relu')
pool1 = max_pool_2d(conv1, 5)

conv2 = conv_2d(pool1, 64, 5, activation='relu')
pool2 = max_pool_2d(conv2, 5)

conv3 = conv_2d(pool2, 128, 5, activation='relu')
pool3 = max_pool_2d(conv3, 5)

conv4 = conv_2d(pool3, 64, 5, activation='relu')
pool4 = max_pool_2d(conv4, 5)

conv5 = conv_2d(pool4, 32, 5, activation='relu')
pool5 = max_pool_2d(conv5, 5)

fully_layer = fully_connected(pool5, 1024, activation='relu')
fully_layer = dropout(fully_layer, 0.5)

cnn_layers = fully_connected(fully_layer, 6, activation='softmax')

cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=LR, loss='binary_crossentropy', name='targets')
model = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=1)

#if (os.path.exists('model.tfl.meta')):
#    model.load('./model.tfl')
#else:



In [ ]:

model.fit({'input': X_train}, {'targets': y_train}, n_epoch=10,
          validation_set=({'input': X_test}, {'targets': y_test}),
           snapshot_step=500,show_metric=True, run_id=MODEL_NAME)
model.save('cnn.tfl')


Training Step: 329  | total loss: 0.71815 | time: 1.153s
| Adam | epoch: 010 | loss: 0.71815 - acc: 0.4379 -- iter: 2048/2061
Training Step: 330  | total loss: 0.71706 | time: 2.197s
| Adam | epoch: 010 | loss: 0.71706 - acc: 0.4457 | val_loss: 0.72170 - val_acc: 0.4207 -- iter: 2061/2061
--


In [ ]:
def create_test_data():
    testing_data=[]
    for img in tqdm(os.listdir(TEST_DIR)):
        print(img)
        path = os.path.join(TEST_DIR, img)
        img_data = cv2.imread(path,1)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        testing_data.append(np.array(img_data))
    np.save('test_data.npy', testing_data)
    return testing_data

test_data = create_test_data()
test = test_data
X_test = np.array([i for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
predictionTest = []
print(X_test.shape)
prediction = model.predict(X_test)
i = 0
for img in tqdm(os.listdir(TEST_DIR)):
  print(img)
  pred = np.argmax(prediction[i])
  predictionTest.append([img,pred])
  i += 1



import csv
with open('/content/CNN.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(["image_name","label"])
    for i in predictionTest:
      writer.writerow(i)
    file.close()


import pandas as pd
df = pd.read_csv('/content/CNN.csv')
print(df)
